## **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


# **Download TensorFlow Model Garden**

In [ ]:
#cd into the TensorFlow directory in your Google Drive and clone the TensorFlow Model Garden repository

%cd '/content/gdrive/MyDrive/TensorFlow'

import tensorflow as tf

#Check for GPU
!nvidia-smi

/content/gdrive/MyDrive/TensorFlow
Tue Sep 14 13:19:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                     

In [ ]:
!git clone https://github.com/tensorflow/models.git 
%cd '/content/gdrive/MyDrive/TensorFlow/models'
!git checkout -f e04dafd04d69053d3733bb91d47d0d95bc2c8199

# **Installation of additional tools**

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

# **Compile Protobuf library**

In [ ]:
#cd into 'TensorFlow/models/research'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'

!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/My Drive/TensorFlow/models/research


# **Set the enviroment**

In [ ]:
import os
import sys

os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"

sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")

# **Installation - setup.py**

In [ ]:
!python setup.py build
!python setup.py install

# **Installation check**

In [ ]:

#Position to 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

**Additional steps:**

1: Copy the “model_main_tf2.py” file from “TensorFlow\models\research\object_detection” and paste it in workspace

2: Copy the “exporter_main_v2.py” file from “TensorFlow\models\research\object_detection” and paste it in workspace

# **Start TensorBoard (start it after finished training)**

In [ ]:
#Position to workspace
%cd '/content/gdrive/My Drive/TensorFlow/workspace'
#Start tensorboarda
%load_ext tensorboard
%tensorboard --logdir=models/[name_of_model]

In [ ]:
#Position to workspace
%cd '/content/gdrive/My Drive/TensorFlow/workspace'

/content/gdrive/My Drive/TensorFlow/workspace


# **Train the model**

In [ ]:
!python model_main_tf2.py --model_dir=models/efficientdet-d0-250slika --pipeline_config_path=models/efficientdet-d0-250slika/pipeline.config

# !python model_main_tf2.py --model_dir=models/"name_of_model" --pipeline_config_path=models/"name_of_model"/pipeline.config

# **Export trained model**

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/efficientdet-d0-250slika/pipeline.config --trained_checkpoint_dir models/efficientdet-d0-250slika/ --output_directory exported-models/my_model_efficientdet-d0-250slika

# !python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/"name_of_model"/pipeline.config --trained_checkpoint_dir models/"name_of_model"/ --output_directory exported-models/my_model_"name_of_model"

# **Evaluation - in my case it doesn't work on Google collab, only locally**

In [ ]:
#Doesn't work on Collab, works when run on PC locally
python model_main_tf2.py \
   --model_dir=models/"name_of_model" \
   --pipeline_config_path=models/"name_of_model"/pipeline.config \
   --checkpoint_dir=models/"name_of_model"



SyntaxError: ignored

# **Testing**

In [ ]:
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/content/gdrive/MyDrive/TensorFlow/workspace/exported-models/my_model_name_of_model/saved_model"

print('Loading the model...', end='')

#Load saved model and start inference
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

# **Testing (Loading label_map)**

In [ ]:
category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/TensorFlow/workspace/annotations/label_map.pbtxt",use_display_name=True)

# **Testing (Image loading)**

In [ ]:
#Position to workspace
%cd '/content/gdrive/My Drive/TensorFlow/workspace'

#image loading
img=['/content/gdrive/My Drive/TensorFlow/workspace/inference/img1.jpg',
     '/content/gdrive/My Drive/TensorFlow/workspace/inference/img4.jpg'
     ]

#print paths of loaded images
print(img)

# **Running the inference**

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))
    
for image_path in img:
  print('Pokretanje detekcija za {}... '.format(image_path), end='')
  image_np=load_image_into_numpy_array(image_path)
    
  input_tensor=tf.convert_to_tensor(image_np)
  input_tensor=input_tensor[tf.newaxis, ...]
    
  detections=detect_fn(input_tensor)
    
  num_detections=int(detections.pop('num_detections'))
  detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}

  detections['num_detections']=num_detections
  detections['detection_classes']= detections['detection_classes'].astype(np.int64)
                
  image_np_with_detections=image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=100,     
          min_score_thresh=.5,      
          agnostic_mode=False)
          
  %matplotlib inline
  plt.figure()
  plt.imshow(image_np_with_detections)
  print('Done')
  plt.show()